<a href="https://colab.research.google.com/github/leighTOR/model-api/blob/main/model_apiv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install the necessary libraries
!pip install flask-ngrok
!pip install torch torchvision  # Install PyTorch if not already installed
!pip install efficientnet_pytorch  # If you're using EfficientNet from this library

In [ ]:
# Step 2: Import necessary libraries
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

In [ ]:
# Step 3: Define the Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Initialize ngrok for public access

In [ ]:
# Step 4: Load your pre-trained EfficientNet model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = EfficientNet.from_name('efficientnet-b0')

    def forward(self, x):
        return self.model(x)

model = MyModel()
model.load_state_dict(torch.load('efficientnet.pt', map_location=torch.device('cpu')))
model.eval()


In [ ]:
# Step 5: Define the predict route
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    input_tensor = torch.tensor(data['input'])  # Convert input to tensor
    input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension if needed

    # Model inference
    with torch.no_grad():
        output = model(input_tensor).numpy().tolist()  # Get the output

    return jsonify({'prediction': output})

In [ ]:
# Step 6: Run the Flask app
if __name__ == '__main__':
    app.run()